In [1]:
import pandas as pd
import requests
import urllib


In [2]:

charts_list = "https://shazam.p.rapidapi.com/charts/list"

charts_list_headers = {
    'x-rapidapi-key': "cd2deb21d4msh076367b8b9327c3p1c1394jsn7657b424467b",
    'x-rapidapi-host': "shazam.p.rapidapi.com"
    }

charts_list_response = requests.request("GET", charts_list, headers=charts_list_headers)



In [3]:

def get_top10_json(list_id):
    url = "https://shazam.p.rapidapi.com/charts/track"

    querystring = {"locale":"en-US","listId":list_id,"pageSize":"20","startFrom":"0"}

    headers = {
        'x-rapidapi-key': "cd2deb21d4msh076367b8b9327c3p1c1394jsn7657b424467b",
        'x-rapidapi-host': "shazam.p.rapidapi.com"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    return response


def get_top10_names(top_10_json):
    tracks_dict = top_10_json['tracks']
    names = []
    for track in tracks_dict:
        names.append((track['title'] + ' - '+ track['subtitle']))
    return names


In [4]:

def get_lat_lon(name):
    f = { 'name' : name}
    formatted_name = urllib.parse.urlencode(f)
    # print(formatted_name)
    response = requests.get("https://api.mapbox.com/geocoding/v5/mapbox.places/{}.json?access_token=pk.eyJ1IjoiaGFyc2gtaiIsImEiOiJja3JraHRmMnEzbnA1MndwOGI2OTY1enNrIn0.Lrx1G8lFIKLt_7OsC6ow7g".format(formatted_name[5:]))
    return response.json()['features'][0]['bbox'][1], response.json()['features'][0]['bbox'][0]


In [5]:

country_list = []
city_list = []
top10 = []
coords_lat = []
coords_lon = []



for country_dict in charts_list_response.json()['countries']:
    for k, city in enumerate(country_dict['cities']):
        if k < 4:
            try:
                lat, lon = get_lat_lon(city['name'])
                coords_lat.append(lat)
                coords_lon.append(lon)
                top_10_json = get_top10_json(city['listid']).json()
                top10.append(get_top10_names(top_10_json))
            except:
                continue
        else:
            break
        print(len(coords_lon))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201


In [6]:
daily_df = pd.DataFrame({"lat": coords_lat, 'lon': coords_lon ,'top10': top10})
daily_df.to_csv('./shazam.csv')

,lat,lon,top10
0,52.338267,13.088359,"[Bad Habits - Ed Sheeran, Love Tonight - Shous..."
1,51.347460,6.894457,"[Bad Habits - Ed Sheeran, Love Tonight - Shous..."
2,48.061596,11.360781,"[Love Tonight - Shouse, Bad Habits - Ed Sheera..."
3,53.394996,8.309567,"[Bad Habits - Ed Sheeran, Love Tonight - Shous..."
4,24.959534,121.458362,"[STAY - The Kid LAROI & Justin Bieber, Bad Hab..."
...,...,...,...
196,54.274974,18.437269,"[Never Going Home - Kungs, I Ciebie też, bardz..."
197,40.680653,27.971306,"[Nabız - Burak Bulut & Kurtuluş Kuş, Into Your..."
198,39.618292,32.234193,"[Into Your Arms - Witt Lowry Feat. Ava Max, Na..."
199,39.995556,28.664827,"[Into Your Arms - Witt Lowry Feat. Ava Max, Na..."


In [12]:
df = pd.read_csv('/Users/harshjhunjhunwala/Desktop/github/spotify/data/shazam.csv')

In [76]:
for row in df.iterrows():
    print(row[1][3], row[1][4])

52.3382670008426 13.0883590415111
51.34746 6.894457
48.061596 11.360781
53.3949960146367 8.30956670062143


In [3]:

import pandas as pd
from pymongo import MongoClient
import json


client = MongoClient('localhost', 27017)

def csv_transfer(csv_path='/Users/harshjhunjhunwala/Desktop/github/spotify/data/shazam.csv', db_name='spotty', coll_name='csv_import', db_url='mongodb://localhost:27017/', db_port=27017):
    """ 
    sending the 'world trends' csv file to mongo_db. This should run once a day to keep the data updated.
    
    Imports a csv file at path csv_name to a mongo colection
    returns: count of the documants in the new collection
    """
    client = MongoClient(db_url, db_port)
    db = client[db_name]
    coll = db[coll_name]
    data = pd.read_csv(csv_path)
    payload = json.loads(data.to_json(orient='records'))
    coll.remove()
    coll.insert(payload)
    return str(coll.count())

In [4]:
# collection = mongo.db.userTracks

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [2]:
csv_transfer

<function __main__.csv_transfer(csv_path='/Users/harshjhunjhunwala/Desktop/github/spotify/data/shazam.csv', db_name='spotty', coll_name='csv_import', db_url='mongodb://localhost:27017/', db_port=27017)>

In [6]:

# for country_dict in charts_list_response.json()['countries']:
#     if country_dict['name'] == 'India':
#         for city in country_dict['cities']:
#             print(city)
#             break
#             if len(city_list) < 4:
#                 country_list.append('India')
#                 city_list.append(city['name'])
#                 top_10_json = get_top10_json(city['listid']).json()
#                 top10.append(get_top10_names(top_10_json))
#                 print(city_list)
#             else:
#                 break
#         break

{'id': '1273294', 'name': 'Delhi', 'countryid': 'IN', 'listid': 'ip-city-chart-1273294'}
